In [1]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from tifffile import imread, TiffWriter
import rigidregistration
from os.path import isfile
from skimage.util import montage

from ncempy.io import ser
%matplotlib widget



In [2]:
from matplotlib import use
#use('ipympl')

In [3]:
rigidregistration

<module 'rigidregistration' from '/Users/noahschnitzer/opt/miniconda3/envs/py373aoi/lib/python3.7/site-packages/rigidregistration-0.2-py3.7.egg/rigidregistration/__init__.py'>

In [4]:
#data_path = '/Users/noahschnitzer/Documents/kem/data/20200124_BSCMO_int_temp/tiffs/'
#dataset = '2213 3.7 Mx STEM_0'
#data = imread(data_path+dataset+'.tif')

#data_path = '/Users/noahschnitzer/Documents/kem/data/From/ieb/20170522_BSCMO_ADF-ABF/'
#dataset = '30_s35_3.6Mx_0.8us_1024_1'
#AA=ser.serReader(data_path+dataset+'.ser')
#data=AA['data']

data_path = '/Users/noahschnitzer/Documents/kem/misc/2021_PARADIM_summer_school/T7_reg/CsBiNb2O7/'
outpath =   data_path+''
dataset = '11_5.1Mx_50x0.5us_1024px_130mmCL_mono50_15pA_70umC2_1'
data = imread(data_path+dataset+'.tif')
data = np.rollaxis(data,0,3)
data.shape

(1024, 1024, 50)

In [5]:
data_slice = data[:,:,5]
#plt.imshow(data_slice, cmap='gray')
fig,ax = plt.subplots()
ax.imshow(data_slice,cmap='gray')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
data = data - np.min(data)
data = data / np.max(data)
#data = np.rollaxis(data,0,3)
data.shape

(1024, 1024, 50)

In [7]:
s=rigidregistration.stackregistration.imstack(data[:,:,:])
s.getFFTs()

working4


In [59]:
#Isotropic

#910 3  4 3  //3.3 5 4
#1p3 4.5  7 6 // 4.8 6 5
#1p8 7 7 6
#2p6 9 14 13
#3p7 6.5 14 13 // 6 9 8
#5p3 9 12 11


# 7p5 9. 8 8 
# 5p1 6.7 7 7
# 3p7 4.6 6 6
# 2p6 7. 7 7
# 1p9 4.7 5 5
# 1p3 3.5 5 5
# 910 2.3 3 3 .......




s.makeFourierMask(mask="hann",n=4)
s.show_Fourier_mask(1,3)


#anisotropic
'''
#5p1
#x =4.5

# 3p7 1k
#x = 3.

#2p6 2k
#x = 1.

#2p6 1k
#x = .8

#1p8 2k
x=1.4

#1p3 2k
#x=1.

#910 2k
#x =.5



n1 = x
n2 = 1.5*x
theta=-7

s.makeFourierMask_eg(n1=n1,n2=n2,theta=np.radians(theta))   # Set the selected mask
s.show_Fourier_mask(i=5,j=9)                                # Display the results
'''

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

'\n#5p1\n#x =4.5\n\n# 3p7 1k\n#x = 3.\n\n#2p6 2k\n#x = 1.\n\n#2p6 1k\n#x = .8\n\n#1p8 2k\nx=1.4\n\n#1p3 2k\n#x=1.\n\n#910 2k\n#x =.5\n\n\n\nn1 = x\nn2 = 1.5*x\ntheta=-7\n\ns.makeFourierMask_eg(n1=n1,n2=n2,theta=np.radians(theta))   # Set the selected mask\ns.show_Fourier_mask(i=5,j=9)                                # Display the results\n'

sigma_guess: sets the initial guess for the standard deviation of the guassian fits, in pixels. This may be estimated simply and quickly by observing the peak widths in the cross correlations or the width of atomic columns in the raw data.

window_radius: sets the size of the region about the brightest pixel which is used to fit a gaussian. Should be set such that neighboring cross correlation peaks are excluded; the window used is a square region of size length 2xwindow_radius+1.

num_peaks: sets how many of the brightest pixels to fit gaussians to. Typically 3-5 are sufficient to handle sampling problems.



In [60]:
findMaxima = 'gf'
s.setGaussianFitParams(num_peaks=5,sigma_guess=5,window_radius=6)
datas,fits,popts,cc,maxima = s.checkGaussianFit(2,17)


posns = maxima[:,::-1]+popts[:,1:3][::1]-s.window_radius
#maxima[:,1]+popts[:,2]-s.window_radius,maxima[:,0]+popts[:,1]-s.window_radius

fig,ax = plt.subplots(1,3)
ax[0].matshow(montage(datas))
ax[0].axis('off')
ax[1].matshow(montage(fits))
ax[1].axis('off')
ax[2].matshow(np.fft.fftshift(cc))
ax[2].plot(posns[:,0],posns[:,1],'r.')
ax[2].axis('off')
#midx = np.argmax(popts[:,6]+popts[:,0])# offsets + amplitudes
midx = np.argmax(2*np.pi*popts[:,0]*popts[:,3]*popts[:,4]+popts[:,6]*np.pi*popts[:,3]*popts[:,4])
ax[2].plot(posns[midx,0],posns[midx,1],'yx')

/Users/noahschnitzer/opt/miniconda3/envs/py373aoi/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if __name__ == '__main__':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [61]:
t0=time()                                                  # Start time 
s.findImageShifts(findMaxima=findMaxima,verbose=True,clippingDimension=None)     # Find shifts.  Set verbose=True to print the correlation status to screen
t=time()-t0                                                # End time
print("Performed {} correlations in {} minutes {} seconds".format(int(s.nz*(s.nz-1)/2),int(t/60),t%60))


Correlating images 0 and 1
Correlating images 0 and 2
Correlating images 0 and 3
Correlating images 0 and 4
Correlating images 0 and 5
Correlating images 0 and 6
Correlating images 0 and 7
Correlating images 0 and 8
Correlating images 0 and 9
Correlating images 0 and 10
Correlating images 0 and 11
Correlating images 0 and 12
Correlating images 0 and 13
Correlating images 0 and 14
Correlating images 0 and 15
Correlating images 0 and 16
Correlating images 0 and 17
Correlating images 0 and 18
Correlating images 0 and 19
Correlating images 0 and 20
Correlating images 0 and 21
Correlating images 0 and 22
Correlating images 0 and 23
Correlating images 0 and 24
Correlating images 0 and 25
Correlating images 0 and 26
Correlating images 0 and 27
Correlating images 0 and 28
Correlating images 0 and 29
Correlating images 0 and 30
Correlating images 0 and 31
Correlating images 0 and 32
Correlating images 0 and 33
Correlating images 0 and 34
Correlating images 0 and 35
Correlating images 0 and 36
C

In [11]:
fig = s.show_Rij(returnfig=True,mask=False)
#fig.axes[0].set_ylim(-1,data.shape[2])
#fig.axes[0].set_xlim(-1,data.shape[2])
#fig.axes[1].set_ylim(-1,data.shape[2])
#fig.axes[1].set_xlim(-1,data.shape[2])

#dir(fig.axes[0])
#dir(fig)
#fig.set_figheight(5)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [57]:
s.set_bad_images([])  

#s.update_Rij_mask()


#s.get_outliers_NN(2)


#s.get_outliers(threshold=10,maxpaths=20)              # Set outlier threshhold

# far off diagonals -> 0
if False:
    for it in range(s.nz):
        for jt in range(s.nz):
            if abs(it-jt)>30:
                s.outlier_mask[it,jt]=False
    s.update_Rij_mask()
    
    
if False:
    for it in range(s.nz):
        s.Rij_mask[it,it]=1


#s.Rij_mask[7,7]=1

fig = s.show_Rij(returnfig=True)






Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [51]:
s.make_corrected_Rij()
s.show_Rij_c()

KeyboardInterrupt: 

In [ ]:
s.get_averaged_image()
s.show() 

In [ ]:
#s.show_report()
save_image_path = outpath+"rI_"+dataset+".tif"
s.save(save_image_path)

In [ ]:
# saving stack as tiff stack
from tifffile import imread, TiffWriter

save_stack_path = outpath+"rS_"+dataset+".tif"

if isfile(save_stack_path):
    raise IOError('File already exists') 
with TiffWriter(save_stack_path, bigtiff=True, append=True) as tif:
    for img_slice in range(0,np.size(s.stack_registered,2)):
        tif.save(np.float32(s.stack_registered[s.xmin:s.xmax,s.ymin:s.ymax,img_slice])) # for cropped
        #tif.save(np.float32(s.stack_registered[:,:,img_slice])) #for uncropped
